In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/train_dataset/caption/tp_test.csv")

In [4]:
data.head(2)

,Unnamed: 0,image_id,prompt,hallucinated_text,source_text,source_metadata,qa_metadata,qa_ids,annotations,id,split
0,64796,2391368,<image>Can you describe the main features of t...,"In this image, there are some boxes that conta...","In this image, there are some boxes contains f...","{'source': 'localized_narratives', 'id': 'sp_2...",[],['001006084'],"{'object': [], 'attribute': [{'attribute': {'n...",caption_4,test
1,64797,2375207,<image>Write a detailed description of the giv...,A man is wearing a white uniform. He is wearin...,A man is wearing a white uniform. He is wearin...,"{'source': 'stanford', 'id': 'sp_30231'}",[],['001021520'],"{'object': [], 'attribute': [{'attribute': {'n...",caption_11,test


In [2]:
data = pd.read_csv('/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/gemini_labeled_40k.csv')

In [3]:
data.head(2)

,image_id,question,answer,candidates,hallucination_candidates,candidates_inx,hallucination_candidates_inx,image_path,question_id
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"['blue', 'hat']",['blue'],"{'blue': [(11, 15)], 'hat': [(4, 7)]}","{'blue': [(11, 15)]}",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,e7f137be-9190-42f0-a1b0-7486e8028fad
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"['hat', 'gray']",['gray'],"{'hat': [(4, 7)], 'gray': [(11, 15)]}","{'gray': [(11, 15)]}",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,c9903799-0507-4102-84fe-3dcd587ae9e0


In [29]:
data["candidates"] = data["candidates"].apply(lambda x: list(set(eval(x))))
data["hallucination_candidates"] = data["hallucination_candidates"].apply(lambda x: list(set(eval(x))))

In [30]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"[blue, hat]",[blue]
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"[hat, gray]",[gray]


In [19]:
import re
import unicodedata
from typing import Dict, Iterable, List, Tuple

def find_word_spans(
    text: str,
    terms: Iterable[str],
    *,
    case_sensitive: bool = False,
    whole_word: bool = False,
    overlapping: bool = False,
    normalize_unicode: bool = True,
) -> Dict[str, List[Tuple[int, int]]]:
    
    if normalize_unicode:
        text = unicodedata.normalize("NFC", text)

    # Prepare result with original (possibly unnormalized) term strings as keys
    results: Dict[str, List[Tuple[int, int]]] = {t: [] for t in terms if t}

    # Build a working list of (orig_term, norm_term)
    work_terms: List[Tuple[str, str]] = []
    for t in terms:
        if not t:
            continue
        t_norm = unicodedata.normalize("NFC", t) if normalize_unicode else t
        work_terms.append((t, t_norm))

    flags = 0 if case_sensitive else re.IGNORECASE

    for orig_term, norm_term in work_terms:
        # Escape to match literal text
        escaped = re.escape(norm_term)

        if whole_word:
            # Word boundaries only at the ends (so phrases are supported)
            # Use (?<!\w) and (?!\w) instead of \b to behave well inside lookaheads for overlapping.
            core = f"(?<!\\w){escaped}(?!\\w)"
        else:
            core = escaped

        if overlapping:
            # Lookahead to capture overlapping starts.
            # Put the core inside a capturing group so we can compute the end index.
            pattern = re.compile(f"(?=({core}))", flags)
            matches = pattern.finditer(text)
            for m in matches:
                start = m.start()
                end = start + len(m.group(1))
                results[orig_term].append((start, end))
        else:
            pattern = re.compile(core, flags)
            matches = pattern.finditer(text)
            for m in matches:
                results[orig_term].append((m.start(), m.end()))

    return results


In [32]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"[blue, hat]",[blue]
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"[hat, gray]",[gray]


In [33]:
candid_inx = [] 
hal_inx = []

for inx, row in data.iterrows():
    spans = find_word_spans(
        text=row["answer"].lower(),
        terms=[i.lower() for i in row["candidates"]])
    candid_inx.append(spans)
    
    hal_spans = find_word_spans(
        text=row["answer"].lower(),
        terms=[i.lower() for i in row["hallucination_candidates"]])
    hal_inx.append(hal_spans)
    

In [34]:
data["candidates_inx"] = candid_inx
data["hallucination_candidates_inx"] = hal_inx

In [35]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates,candidates_inx,hallucination_candidates_inx
0,2317351.jpg,Describe the color of the hat.,The hat is blue.,"[blue, hat]",[blue],"{'blue': [(11, 15)], 'hat': [(4, 7)]}","{'blue': [(11, 15)]}"
1,2317351.jpg,Describe the color of the hat.,The hat is gray.,"[hat, gray]",[gray],"{'hat': [(4, 7)], 'gray': [(11, 15)]}","{'gray': [(11, 15)]}"


In [36]:
data.to_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/gemini_labeled_40k.csv", index=False)

In [58]:
sampled_data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/instruct/original_sampled_20k.csv")
total_data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/train_dataset/instruct/tp_train.csv")

In [59]:
sampled_ids = sampled_data["id"].to_list()

In [60]:
ramian_data = total_data[~total_data["id"].isin(sampled_ids)]

In [5]:
data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/train_dataset/caption/tp_test.csv")

In [6]:
ramian_data = data

In [7]:
ramian_data.head(5)

,Unnamed: 0,image_id,prompt,hallucinated_text,source_text,source_metadata,qa_metadata,qa_ids,annotations,id,split
0,64796,2391368,<image>Can you describe the main features of t...,"In this image, there are some boxes that conta...","In this image, there are some boxes contains f...","{'source': 'localized_narratives', 'id': 'sp_2...",[],['001006084'],"{'object': [], 'attribute': [{'attribute': {'n...",caption_4,test
1,64797,2375207,<image>Write a detailed description of the giv...,A man is wearing a white uniform. He is wearin...,A man is wearing a white uniform. He is wearin...,"{'source': 'stanford', 'id': 'sp_30231'}",[],['001021520'],"{'object': [], 'attribute': [{'attribute': {'n...",caption_11,test
2,64798,2351441,<image>Analyze the image in a comprehensive an...,there are three people on skis. they are treck...,there are three people on skis. they are treck...,"{'source': 'stanford', 'id': 'sp_26758'}",[],"['00116341', '00116312']","{'object': [{'obj': {'name': 'motorcycle', 'ch...",caption_38,test
3,64799,2382256,<image>Explain the visual content of the image...,This is the dog sitting on the floor. Here is ...,This is the dog sitting on the floor. Here is ...,"{'source': 'localized_narratives', 'id': 'sp_2...",[],['00141230'],"{'object': [], 'attribute': [{'attribute': {'n...",caption_42,test
4,64800,2386140,<image>What's happening in the scene?,A white kite with a green streamer is flying i...,A white kite with a green streamer is flying i...,"{'source': 'stanford', 'id': 'sp_32416'}",[],['00149629'],"{'object': [], 'attribute': [{'attribute': {'n...",caption_47,test


In [8]:
ramian_data["prompt"] = ramian_data["prompt"].apply(lambda x: x.replace("<image>",""))

In [9]:
ramian_data = ramian_data[["image_id", "prompt", "hallucinated_text", "annotations"]]

In [10]:
ramian_data.head(2)

,image_id,prompt,hallucinated_text,annotations
0,2391368,Can you describe the main features of this ima...,"In this image, there are some boxes that conta...","{'object': [], 'attribute': [{'attribute': {'n..."
1,2375207,Write a detailed description of the given image.,A man is wearing a white uniform. He is wearin...,"{'object': [], 'attribute': [{'attribute': {'n..."


In [11]:
all_hal_cand = []

for anot in ramian_data["annotations"]:
    res = eval(anot)
    hal_cand = []
    for val in res.values():
        for i in val:
            for j in list(i.values()):
                token =  j['name']
                hal_cand.append(token)
    all_hal_cand.append(list(set(hal_cand)))

In [12]:
ramian_data["hallucination_candidates"] = all_hal_cand
ramian_data["candidates"] = [[i] for i in ramian_data["hallucinated_text"]]


/tmp/ipykernel_691122/1222014100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["hallucination_candidates"] = all_hal_cand
/tmp/ipykernel_691122/1222014100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["candidates"] = [[i] for i in ramian_data["hallucinated_text"]]


In [13]:
ramian_data.drop(columns=["annotations"], inplace=True)

/tmp/ipykernel_691122/643254796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data.drop(columns=["annotations"], inplace=True)


In [14]:
ramian_data.head(2)

,image_id,prompt,hallucinated_text,hallucination_candidates,candidates
0,2391368,Can you describe the main features of this ima...,"In this image, there are some boxes that conta...","[wicker, tray]","[In this image, there are some boxes that cont..."
1,2375207,Write a detailed description of the given image.,A man is wearing a white uniform. He is wearin...,"[plastic, baseball bat]",[A man is wearing a white uniform. He is weari...


In [15]:
ramian_data.columns = ["image_path", "question", "answer", "hallucination_candidates", "candidates"]


In [16]:
ramian_data.head(2)

,image_path,question,answer,hallucination_candidates,candidates
0,2391368,Can you describe the main features of this ima...,"In this image, there are some boxes that conta...","[wicker, tray]","[In this image, there are some boxes that cont..."
1,2375207,Write a detailed description of the given image.,A man is wearing a white uniform. He is wearin...,"[plastic, baseball bat]",[A man is wearing a white uniform. He is weari...


In [20]:
hal_inx = []

for inx, row in ramian_data.iterrows():
    hal_spans = find_word_spans(
        text=row["answer"].lower(),
        terms=[i.lower() for i in row["candidates"]])
    hal_inx.append(hal_spans)
    

In [21]:
ramian_data["candidates_inx"] = hal_inx

/tmp/ipykernel_691122/4104138947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["candidates_inx"] = hal_inx


In [22]:
ramian_data.head(2)

,image_path,question,answer,hallucination_candidates,candidates,candidates_inx
0,2391368,Can you describe the main features of this ima...,"In this image, there are some boxes that conta...","[wicker, tray]","[In this image, there are some boxes that cont...","{'in this image, there are some boxes that con..."
1,2375207,Write a detailed description of the given image.,A man is wearing a white uniform. He is wearin...,"[plastic, baseball bat]",[A man is wearing a white uniform. He is weari...,{'a man is wearing a white uniform. he is wear...


In [23]:
ramian_data["image_path"] = ramian_data["image_path"].apply(lambda x: str(x)+ ".jpg")

/tmp/ipykernel_691122/308830470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ramian_data["image_path"] = ramian_data["image_path"].apply(lambda x: str(x)+ ".jpg")


In [24]:
ramian_data.head(2)

,image_path,question,answer,hallucination_candidates,candidates,candidates_inx
0,2391368.jpg,Can you describe the main features of this ima...,"In this image, there are some boxes that conta...","[wicker, tray]","[In this image, there are some boxes that cont...","{'in this image, there are some boxes that con..."
1,2375207.jpg,Write a detailed description of the given image.,A man is wearing a white uniform. He is wearin...,"[plastic, baseball bat]",[A man is wearing a white uniform. He is weari...,{'a man is wearing a white uniform. he is wear...


In [25]:
ramian_data.shape

(3273, 6)

In [26]:
from uuid import uuid4

img_folder = "/Data2/Arun-UAV/NLP/vision_halu/visual_genome/target_images/"

ramian_data = ramian_data.rename(columns={"image_path": "image_id"})
ramian_data["image_path"] = ramian_data["image_id"].apply(lambda x: img_folder + x)
ramian_data["question_id"] = [str(uuid4()) for _ in range(len(ramian_data))]

In [27]:
ramian_data.head(2)

,image_id,question,answer,hallucination_candidates,candidates,candidates_inx,image_path,question_id
0,2391368.jpg,Can you describe the main features of this ima...,"In this image, there are some boxes that conta...","[wicker, tray]","[In this image, there are some boxes that cont...","{'in this image, there are some boxes that con...",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,24ff2006-5782-4efc-a33c-d20a3e601e6c
1,2375207.jpg,Write a detailed description of the given image.,A man is wearing a white uniform. He is wearin...,"[plastic, baseball bat]",[A man is wearing a white uniform. He is weari...,{'a man is wearing a white uniform. he is wear...,/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,42ef9f38-105d-40fb-a696-5caec821462b


In [28]:
ramian_data = ramian_data.drop(["candidates", "candidates_inx"], axis=1)

In [29]:
ramian_data.head(2)

,image_id,question,answer,hallucination_candidates,image_path,question_id
0,2391368.jpg,Can you describe the main features of this ima...,"In this image, there are some boxes that conta...","[wicker, tray]",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,24ff2006-5782-4efc-a33c-d20a3e601e6c
1,2375207.jpg,Write a detailed description of the given image.,A man is wearing a white uniform. He is wearin...,"[plastic, baseball bat]",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,42ef9f38-105d-40fb-a696-5caec821462b


In [ ]:
ramian_data.to_csv("/Data2/Arun-UAV/NLP/vision_halu/benchmarks/holoc/test_caption_3200.csv", index=False)

In [30]:
ramian_data.shape

(3273, 6)